**TASKS**:
- use the initialize() function to get the basis set instances and the jkfactory object 
- get overlap matrix (S) in the AO basis (basis_gen)
- set the U matrix -> block orthogonalize the overlap

Note that :

The density matrix in the AO basis and the corresponding matrix in the BO basis are related by
$$ D^{ao} = U \tilde{D} U^{\dagger} $$
Morover $ U^{-1} D^{ao} (U^{\dagger})^{-1} = \tilde{D} $. $U$ is not orthogonal ($U^{-1} != U^{\dagger}$).

Correspondingly the matrix representation of a given operator in the BO basis can be obtained from the AO basis representation as:
$$ \tilde{O} = U^{\dagger} O^{ao} U $$

In [1]:
### 
import os
import sys
import psi4
import numpy as np

sys.path.insert(0, "./common")
modpaths = os.environ.get('MODS_PATH')

if modpaths is not None :
    for path in modpaths.split(";"):
        sys.path.append(path)
from scf_run import run
from init_run import initialize
from Fock_helper import fock_factory
print("test mixed basis / functionals")
func_high = 'b3lyp'
func_low = 'blyp'
bset,bsetH, molelecule_str, psi4mol, wfn, jkobj = initialize(False,'DIRECT','3-21G;O1:def2-svp;H1:def2-svp','1',\
                                                             'H2O1.xyz',func_high,func_low,eri='nofit',debug=True)

mints = psi4.core.MintsHelper(bset)
#I = np.array(mints.ao_eri())

H = np.array(mints.ao_kinetic())+ np.array(mints.ao_potential())
S = np.array(mints.ao_overlap())
numbas = bset.nbf() #get the number of basis function belonging to bset (the low-level-theory basis)

nbfA = bsetH.nbf() #get the number of basis function belonging to bsetH (the high-level-theory basis)

#make U matrix for blend basis(cc-pvdz+3-21G)
U = np.eye(numbas)
S11=S[:nbfA,:nbfA]
S11_inv=np.linalg.inv(S11)
S12 =S[:nbfA,nbfA:]
P=np.matmul(S11_inv,S12)
U[:nbfA,nbfA:]=-1.0*P

#S block orthogonal
Stilde= np.matmul(U.T,np.matmul(S,U))


mtest=np.zeros((nbfA,(numbas-nbfA)))
print("AB block of overlap (in BO basis) is zero: %s" %(np.allclose(Stilde[:nbfA,nbfA:],mtest)))


test mixed basis / functionals
High Level functional : b3lyp

Low Level functional : blyp

number of fragmemts: 2

functions in general basis: 21
functions in subsys1: 19

Size of the ERI tensor will be 0.00 GB.
eri n. axis: 4
eri is instance: <class 'numpy.ndarray'>

jkfactory -> eri tensor provided, real-time HF exch required: False
AB block of overlap (in BO basis) is zero: True


---------------------------------------------------------
**Use fock_factory to get the Fock in BO basis**

A fock_factory instance can be initialized as:

fock_engine = **fock_factory**(jk_fact,Hmat,ovapm,*funcname*=None,*basisobj*=None,*exmodel*=0)

input: 

- jk_fact: the jk object obtained from the "initialize" function
- Hmat: the $ H_{core}$ matrix in the AO basis
- ovapm: the basis function overlap matrix. Only needed when we work in the density matrix framework, i.e
  forming $ SDS $, and diagonalizing the "density operator" we get natural orbitals. ovapm can be either $\tilde{S}$   or $S^{ao}$ depending on the basis we are working, respectively, BO or AO basis
- funcname: the functional corresponding to the low level theory, when BO embedding is intended. Otherwise is the     usual functional in a supermolecular calculation.
- basis object: same as funcname
- exmodel : exchange model, 0 is assumed, see Manby-Miller and Parkhill papers

fock_factory has many methods:

- **get_bblock_Fock**(self,*Cocc*=None,*Dmat*=None,*func_acc*=None,*basis_acc*=None,*U*=None,*return_ene*=False). This function can accept either the matrix containing MO coefficients or the density matrix (in the appropriate basis). **basis_acc** and **func_acc** denote the 'high-level-theory' basis object and functional respectively. $U$ has to be passed as numpy.ndarray 
- **get_Fock**(self,*Cocc*=None,*Dmat*=None,*return_ene*=False). This method can be used to get the usual supermolecular Fock matrix corresponding to $C_{occ}$/$D_{mat}$, *funcname* and *basisobj* (see the intialization)




**TASK**
- test the b3lyp-in-blyp case
- perform a full scf calculation using scf_run.run()
- run() functions declaration: 

**run**(jkclass,embmol,bset,bsetH,guess,func_h,func_l,exmodel,wfn)
- jkclass : True|False, enable native Psi4 JK class 
- embmol  : Psi4 molecule object
- bset    : total basis set object
- bsetH   : 'high-level-theory' fragment basis set object
- guess   : specify the type of initial guess 'GS'|'SAD' (i.e ground state/sum of atomic densities)
- func_h  : 'high-level-theory' fragment functional
- func_l  : 'low-level-theory' fragment functional
- exmodel : exact exchange model, default = 0
- wfn     : Psi4 wavefunction object (correspondig to 'func_l' run

In [2]:
Cocc = np.array(wfn.Ca_subset('AO','OCC'))

try:
    U_inv = np.linalg.inv(U)
except np.linalg.LinAlgError:
    print("Error in numpy.linalg.inv of inputted matrix")

Cocc = np.matmul(U_inv,Cocc)
Dinput = np.matmul(Cocc,Cocc.T)
fockbase = fock_factory(jkobj,H,Stilde,funcname=func_low,basisobj=bset) #Stilde as overlap matrix
F_bblock = fockbase.get_bblock_Fock(Dmat=Dinput,func_acc=func_high,basis_acc=bsetH,U=U)

print("F(BO) dim: %i,%i\n" % (F_bblock.shape[0],F_bblock.shape[1]))



F(BO) dim: 21,21



In [3]:
#define the embopt dataclass
import fde_utils
from fde_utils import embedoption
pyembopt = embedoption
pyembopt.nofde = True #skip FDE=True, normally passed from input as --fde -> pyembopt.nofde = not args.fde 

In [4]:

res, wfnBO = run(jkobj,psi4mol,bset,bsetH,'GS',func_high,func_low,0,wfn,pyembopt)


Overlap_AA in BO is Overlap_AA in AO: True
Overlap_AB in BO is zero: True
using EX model: ......... 0


Number of occupied orbitals: 5

using as guess the density from the low level theory hamiltonian

Start SCF iterations:


SCF Iteration   1: Energy = -76.3472224073081094   dE = -7.63472E+01   dRMS = 1.39404E-03
SCF Iteration   2: Energy = -76.3476167551444433   dE = -3.94348E-04   dRMS = 3.48690E-04
SCF Iteration   3: Energy = -76.3476174444848397   dE = -6.89340E-07   dRMS = 3.70139E-04
SCF Iteration   4: Energy = -76.3476390034645220   dE = -2.15590E-05   dRMS = 3.99263E-05
SCF Iteration   5: Energy = -76.3476392634172498   dE = -2.59953E-07   dRMS = 3.39390E-06
SCF Iteration   6: Energy = -76.3476392652796250   dE = -1.86238E-09   dRMS = 3.51387E-07
SCF Iteration   7: Energy = -76.3476392652982696   dE = -1.86446E-11   dRMS = 6.33270E-08
SCF Iteration   8: Energy = -76.3476392652989375   dE = -6.67910E-13   dRMS = 7.55661E-09
Total time for SCF iterations: 4.327 seconds 


Final 

In [5]:
#compare the b3lyp-in-blyp orbital enegies/total energy to pure blylp
eps_a = np.array( wfn.epsilon_a() )
ndocc = wfnBO['ndocc']
print('Doubly Occupied: (%s / %s-in-%s)\n' % (func_low,func_high,func_low))
for k in range(ndocc):
    print('%iA : %.6f | %.6f' %(k+1,eps_a[k],wfnBO['epsilon_a'][k]))

print('Virtual:\n')

for k in range(ndocc,numbas):
    print('%iA : %.6f | %.6f'% (k+1,eps_a[k],wfnBO['epsilon_a'][k]))


Doubly Occupied: (blyp / b3lyp-in-blyp)

1A : -18.757985 | -19.112944
2A : -0.885721 | -0.982307
3A : -0.458443 | -0.516714
4A : -0.297654 | -0.362454
5A : -0.222960 | -0.290447
Virtual:

6A : 0.033236 | 0.056658
7A : 0.126856 | 0.152047
8A : 0.556727 | 0.603089
9A : 0.821197 | 0.848560
10A : 0.866244 | 0.921669
11A : 0.882004 | 0.929445
12A : 1.018074 | 1.060609
13A : 1.092564 | 1.133453
14A : 1.388193 | 1.436425
15A : 1.519533 | 1.561976
16A : 2.173896 | 2.242830
17A : 2.536124 | 2.608437
18A : 2.836908 | 2.908813
19A : 2.911465 | 2.988679
20A : 2.982020 | 3.041398
21A : 3.601674 | 3.688129


In [6]:
#get Density matrix and Fock(0) from wfnBO

tD = wfnBO['Dmtx']
tFock = wfnBO['Fock']
U = wfnBO['Umat']
C = wfnBO['Ccoeff'] 
Stilde = wfnBO['Ovap'] 
# get rt params
from rt_mod import set_params
pulse_opts, calc_params = set_params('./ipynotebook/input.inp')
delta_t = calc_params['delta_t']


In [7]:
#initialize the mints object
mints = psi4.core.MintsHelper(bset)
#intialize fock_factory
from Fock_helper import fock_factory
Hcore = np.asarray(mints.ao_potential()) + np.asarray(mints.ao_kinetic())

fock_base = fock_factory(jkobj,Hcore,Stilde, \
                            funcname=func_low,basisobj=bset,exmodel=0)


In [8]:
#dip_mat is transformed in the BO basis
dipole=mints.ao_dipole()
direction = 2 # i.e z-dir
dip_mat=np.matmul(U.T,np.matmul(np.array(dipole[direction]),U))


class **real_time**():

initialization params:

- *Dinit* : intial density matrix (t=0)
- *Fock_init* : guess Fock matrix
- *fock_factory* : fock factory object
- *ndocc* : number of doubly occupied MO
- *basis* : supermolecular basis set object
- *Smat* : overlap matrix (representation dependent, can be either BO or AO)
- *pulse_opts* : pulse paramter dictionary
- *delta_t* : time-step
- *Cmat* : matrix used get the relevant quantities to the propagation basis Cmat = $C_{coeff}$ {or $S^{-1/2}$}
- *dipmat* : matrix representation of dipole operator (either on BO or AO basis)
- out_file=sys.stderr :output err file

the following parameters have defaults values, if not speciefied  the real-time object 
should lead to the same results of supermolecular calculation.

- basis_acc = None : the small 'high-level-theory' basis set object
- func_acc=None : the 'high-level-theory' functional
- U=None : the U matrix -> $\tilde{S} = U^{T}SU$
- local_basis=False : Bool, enable local basis (RLMO localization scheme)
- exA_only=False : Bool , restrict the excitation to basis_acc subblock elements (nbfA $\times$ nbfA) of dipole
- occlist=None : a list containting identifiers of occupied MOs, to be used in occ->virt excitation
- virtlist=None : a list containting identifiers of virtual MOs


In [9]:
from rt_base import real_time
rt_prop = real_time(tD, tFock, fock_base, ndocc, bset, Stilde, pulse_opts, delta_t, C, dip_mat,\
            out_file=sys.stderr,  basis_acc = bsetH, func_acc=func_high,U=U,local_basis=False,\
                                             exA_only=False,occlist=None, virtlist=None)

In [10]:
Dp_init , Da = rt_prop.init_boost()

Perturb density with analytic delta
Dipole matrix is transformed to the MO basis

Local basis: False



In [11]:
rt_prop()

In [12]:
print(rt_prop.iter_num())

1


In [13]:
for k in range(rt_prop.iter_num(),10):
    rt_prop()

In [14]:
# get the the density matrix at step 10 (AO)
# get a corresponding Fock, J and K ..
# set a new jk factory using native PS4 jkclass and fitted eri to get the imaginary part
Dmat_test_bo = rt_prop.get_Dmat('AO')  # the density matrix is expressed in a non-orthogonal basis (either AO or BO)
#trace = np.matmul(Dmat_test,Stilde)
#print(" trace of DS in BO %.5f\n" % np.trace(trace))
#in the AO basis
Dmat_test = np.matmul(U,np.matmul(Dmat_test_bo,U.T))
trace = np.matmul(Dmat_test,S)
print(" trace of DS in AO %.5f\n" % np.trace(trace))

 trace of DS in AO 5.00000



/tmp/ipykernel_23273/696474344.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  print(" trace of DS in AO %.5f\n" % np.trace(trace))


In [15]:
Fock_test = fock_base.get_bblock_Fock(None,Dmat_test_bo,func_acc=func_high,basis_acc=bsetH,U=U)
J_ao = fock_base.J(None,Dmat=Dmat_test)
K_ao = fock_base.K(None,Dmat=Dmat_test)

In [16]:
# re init with different setup
dum1,dum2, dum3, dum4, dum5, jkfact_pure = initialize(True,'DIRECT','3-21G;O1:def2-svp;H1:def2-svp','1','H2O1.xyz',\
                   func_high,func_low,eri='fit',rt_HF_iexch=True)

High Level functional : b3lyp

Low Level functional : blyp

number of fragmemts: 2

functions in general basis: 21
functions in subsys1: 19

Size of the 3-index ERI tensor will be 0.00 GB.
eri shape [113,21,21]
eri n. axis: 3
eri is instance: <class 'numpy.ndarray'>

using DIRECT scf and JK class

jkfactory -> eri tensor provided, real-time HF exch required: True
jkfactory -> JK psi4 class being in use!!


In [17]:
fock_base_pure = fock_factory(jkfact_pure,Hcore,Stilde, \
                            funcname=func_low,basisobj=bset,exmodel=0)

In [18]:
Fock_test_pure = fock_base_pure.get_bblock_Fock(None,Dmat_test_bo,func_acc=func_high,basis_acc=bsetH,U=U)

In [19]:
print(np.allclose(Fock_test_pure,Fock_test))
print(np.max(np.abs(Fock_test_pure-Fock_test)))

False
1.1090800354528287e-06


In [20]:
#get Cocc from Dmat_test (AO)
import scipy
ndocc = int(np.rint(np.trace(np.matmul(Dmat_test,S)).real))
#    print(ndocc)
#    diagonalize to get Cocc
den_op = np.matmul(S,np.matmul(Dmat_test.real,S))
w, v  = scipy.linalg.eigh(den_op,S)
idx_w = w.argsort()[::-1]
w = w[idx_w]
v = v[:,idx_w]
Cocc = v[:,:ndocc]
check =  np.matmul(Cocc,Cocc.T)
print(np.allclose(check,Dmat_test.real))

True


In [21]:
J_internal_ao = fock_base_pure.get_Fterm('coul')

In [22]:
J_pure_ao = fock_base_pure.J(Cocc,Dmat_test)
K_pure_ao = fock_base_pure.K(Cocc,Dmat_test)

In [23]:
print(np.allclose(J_pure_ao,J_internal_ao))
print(np.allclose(J_ao,J_internal_ao))

print(np.allclose(K_ao.real,K_pure_ao.real))

True
True
True


In [24]:
np.savetxt('K_pure.out',K_pure_ao.imag)

In [25]:
np.savetxt('K_eri.out',K_ao.imag)
np.savetxt('K_eri_real.out',K_ao.real)

In [26]:
np.max(np.abs(K_ao.real-K_pure_ao.real))

1.956579342987652e-11

In [27]:
np.max(np.abs(K_ao.imag-K_pure_ao.imag))

5.561713924519798e-06

In [28]:
print(np.trace(np.matmul(K_ao,Dmat_test)))
print(np.trace(np.matmul(K_pure_ao,Dmat_test)))

(8.956203853260433+5.729375747974292e-15j)
(8.956203853169303+5.729364313029504e-15j)
